In [1]:
import sys
sys.path.append("..")
%cd ..

c:\Users\frede\OneDrive - Danmarks Tekniske Universitet\DTU\6. Semester\Bachelor2021-1


In [2]:
from DataPrep.load_data import load_data
import pyforest

In [3]:
df = pd.read_csv("data\createdDat\Data_Oskar\df_11052021.csv")
plug = pd.read_csv("data\createdDat\Data_Oskar\plugdf_11052021.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
plug[plug["Label"]==5].describe()

,Label,Plugs_raw,Number of NEMA 5-20R_raw,Plugs,Number of NEMA 5-20R
count,1825.0,1825.0,1825.0,1825.0,1825.0
mean,5.0,6.0,3.0,6.0,3.0
std,0.0,0.0,0.0,0.0,0.0
min,5.0,6.0,3.0,6.0,3.0
25%,5.0,6.0,3.0,6.0,3.0
50%,5.0,6.0,3.0,6.0,3.0
75%,5.0,6.0,3.0,6.0,3.0
max,5.0,6.0,3.0,6.0,3.0


In [5]:
def get_censored(df):
    plug = pd.read_csv("data\createdDat\Data_Oskar\plugdf_11052021.csv")

    df = df.drop(columns=["Unnamed: 0","Latitude","Longitude","CenterLat","CenterLon", "Plug Type"])
    df["Start Date"] = pd.to_datetime(df["Start Date"]) 
    test = pd.DataFrame(columns=list(df.columns))
    censors = []
    for label in range(8):
        maks = plug["Date"][plug["Label"]==label][plug[plug["Label"]==label]["Plugs"]==plug[plug["Label"]==label]["Plugs"].max()]
        intercept = maks[maks.index[0]]
        macs_until_intercept = df[(df["Label"]==label) & (df["Start Date"].dt.date<pd.to_datetime(intercept))]["Station Name"].unique()

        if label==5:
            macs_until_intercept = df["Station Name"][df["Label"]==label].unique()

        #Label 2 skal ikke censores kun 4 dage uden fuld kraft
        liste = df[df["Label"]==label]["Station Name"].isin(macs_until_intercept)

        censors.append(intercept)

        test = test.append(df[(df["Label"]==label) & (liste)]) 
    
    return test, censors
    



In [6]:
test, censors = get_censored(df)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
def downsampling_to_days(df):
    labels = df["Label"].unique()
    dfClean2 = pd.DataFrame(columns = df.columns[1:])
    for label in labels:
        d = df[df["Label"] == label].resample("D", on="Start Date").agg({'Charging Time (mins)':'sum','Total Duration (mins)':'sum', 'Energy (kWh)':'sum', 'Fee (USD)':'sum', 'MAC Address':'min', 'Station Name':'min'}).fillna('')
        d["Label"] = label
        
        dfClean2 = dfClean2.append(d)
    
    dfClean2 = dfClean2.reset_index()
    dfClean2 = dfClean2.rename(columns = {"index":"Start Date"})
    
    new_col_order = ['Start Date','Label','Charging Time (mins)','Total Duration (mins)', 'Energy (kWh)', 'Fee (USD)', 'MAC Address', 'Station Name']
    dfClean3 = dfClean2[new_col_order]
    return dfClean3

In [8]:
df_down = downsampling_to_days(test)
df_down = df_down.dropna(axis=1)
df_down=df_down.set_index("Start Date")

<IPython.core.display.Javascript object>

In [9]:
for i in range(8):
    df_down.loc[lambda x: (x["Label"]==i) & (x.index>pd.to_datetime(censors[i])), "Censored"] = 1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
df_down = df_down.fillna(0)

In [14]:
df_down.to_csv("data\createdDat\Censorship_scheem\Plugs_censord.csv")